In [ ]:
from scipy.io import wavfile
import scipy.io
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import tftb
import librosa, librosa.display
import os

In [ ]:
def read_audio(audio) :
    audio_file_path = audio
    samplerate, data = wavfile.read(audio_file_path)
    length = data.shape[0] / samplerate
    return samplerate,data,length

def signal_show(data,samplerate):
    length = data.shape[0] / samplerate
    time = np.linspace(0., length, data.shape[0])
    plt.plot(time, data)

    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    #plt.show()

def spectrogram(data,size,length,samplerate) :
    NFFT=1024
    #print("Size ",size)
    #print("Length ",length)
    subsampling_rate= size/length
    #print("Subsampling rate: ",subsampling_rate)
    ts = np.arange(size)* (1/subsampling_rate)
    dt=1/subsampling_rate

    window_hann = np.hanning(1024)

    #plt.figure(figsize=(15,8))
    spectrum,freqs,time,im=plt.specgram(data, Fs=subsampling_rate, cmap="jet", NFFT=NFFT
                                        ,noverlap =128,window=window_hann,detrend="mean" )

    spectrum_db = 20 * np.log10(spectrum / np.mean(spectrum))

    one_d_arr = spectrum_db.flatten()
    sorted_arr = np.sort(one_d_arr)
    percentile_index = int(0.7 * len(sorted_arr))
    threshold = sorted_arr[percentile_index-1]
    spectrum_db[spectrum_db <= threshold] = np.min(spectrum_db)


    freq=np.linspace(0,subsampling_rate/2,num=spectrum_db.shape[0])
    #print("Maximum freq: ",np.max(freq))
    df = freq[1]-freq[0]

    #print("spectrum_size",spectrum.shape)
    plt.imshow(
        spectrum_db,
        extent=(ts[0], ts[-1]+dt, freqs[0], freqs[-1]+df),
        origin="lower",
        cmap="jet",
        aspect="auto",vmin=np.min(spectrum_db), vmax=np.max(spectrum_db)
    )
    plt.colorbar()
    plt.title('Spectrogram of tapir')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    return spectrum_db,np.min(spectrum_db),np.max(spectrum_db)

In [ ]:
def spectrogram_afterfilter(data,size,length,samplerate,min_db,max_db) :
    NFFT=1024
    #print("Size ",size)
    #print("Length ",length)
    subsampling_rate= size/length
    print("Subsampling rate: ",subsampling_rate)
    ts = np.arange(size)* (1/subsampling_rate)
    dt=1/subsampling_rate

    window_hann = np.hanning(1024)

    #plt.figure(figsize=(15,8))
    spectrum,freqs,time,im=plt.specgram(data, Fs=subsampling_rate, cmap="jet", NFFT=NFFT
                                        ,noverlap =128,window=window_hann,detrend="mean" )

    #make sure there is no zero in the array
    smallest_positive = np.amin(spectrum[spectrum > 0])
    spectrum=np.where(spectrum <= 0, smallest_positive, spectrum)

    spectrum_db = 20 * np.log10(spectrum / np.mean(spectrum))

    one_d_arr = spectrum_db.flatten()
    sorted_arr = np.sort(one_d_arr)
    percentile_index = int(0.9 * len(sorted_arr))
    threshold = sorted_arr[percentile_index-1]
    spectrum_db[spectrum_db <= threshold] = np.min(spectrum_db)


    freq=np.linspace(0,subsampling_rate/2,num=spectrum_db.shape[0])
    #print("Maximum freq: ",np.max(freq))
    df = freq[1]-freq[0]

    print("spectrum_size",spectrum.shape)
    plt.imshow(
        spectrum_db,
        extent=(ts[0], ts[-1]+dt, freqs[0], freqs[-1]+df),
        origin="lower",
        cmap="jet",
        aspect="auto",vmin=min_db, vmax=max_db
    )
    """plt.colorbar()
    plt.title('Spectrogram of tapir')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')"""

    plt.axis('off')
    plt.margins(y=0,x=0)


    """plotpath='/content/drive/MyDrive/FYP/Spectrogram/Tapir/Tapir_zoo_png/01_01_ch0_zoo.png'
    plt.savefig(plotpath, bbox_inches="tight",pad_inches = 0)"""
    return spectrum_db

In [ ]:
def downsampling(data,samplerate,new_samplerate):
    length = data.shape[0] / samplerate
    new_sample=int(new_samplerate*length)
    new_data = signal.resample(data, new_sample)
    print(new_data.shape)
    return new_data


In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, sr, order):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def filter_signal(data, sr, lowcut, highcut, order):
    b, a = butter_bandpass(lowcut, highcut, sr, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
from tftb.processing.base import BaseTFRepresentation
class WignerVilleDistribution(BaseTFRepresentation):

    name = "wigner-ville"

    def run(self):
        tausec = round(self.n_fbins / 2.0)
        winlength = tausec - 1
        taulens = np.min(np.c_[np.arange(self.signal.shape[0]),
                               self.signal.shape[0] - np.arange(self.signal.shape[0]) - 1,
                         winlength * np.ones(self.ts.shape)], axis=1)
        conj_signal = np.conj(self.signal)
        for icol in range(self.ts.shape[0]):
            taumax = taulens[icol]
            tau = np.arange(-taumax, taumax + 1).astype(int)
            indices = np.remainder(self.n_fbins + tau, self.n_fbins).astype(int)
            self.tfr[indices, icol] = self.signal[icol + tau] * \
                conj_signal[icol - tau]
                
            if (icol <= self.signal.shape[0] - tausec-1) and (icol >= tausec + 1):
                self.tfr[tausec, icol] = self.signal[icol + tausec] * \
                    np.conj(self.signal[icol - tausec]) + \
                    self.signal[icol - tausec] * conj_signal[icol + tausec]
        self.tfr = np.fft.fft(self.tfr, axis=0)
        self.tfr = np.real(self.tfr)
        self.freqs = 0.5 * np.arange(self.n_fbins, dtype=float) / self.n_fbins
        return self.tfr, self.ts, self.freqs


    def plot(self, kind='cmap', threshold=0.05, sqmod=False, **kwargs):
        scale = kwargs.pop("scale", "linear")
        if scale == "log":
            maxi = np.amax(self.tfr)
            mini = max(np.amin(self.tfr), maxi * threshold)
            levels = np.logspace(np.log10(mini), np.log10(maxi), 65)
            kwargs['levels'] = levels
        if sqmod:
            self.tfr = np.abs(self.tfr) ** 2
        else:
            self.tfr = np.abs(self.tfr)
        _threshold = np.amax(self.tfr) * threshold
        self.tfr[self.tfr <= _threshold] = 0.0
        super(WignerVilleDistribution, self).plot(kind=kind, threshold=threshold,
                                                  **kwargs)

In [ ]:
def WVD(data,samplerate,new_size,length):
    #no slice , sqmod = true , fwd obtain from linespace
    sample=data.shape[0]
    NFFT=18
    fbin=np.arange(NFFT) * (samplerate/NFFT)
    subsampling_rate = new_size/length
    ts = np.arange(sample) * (1/subsampling_rate)  # times
    dt= (1/subsampling_rate)

    #wvd = tftb.processing.WignerVilleDistribution(data, timestamps=ts,n_fbins=128)
    wvd =WignerVilleDistribution(data, timestamps=ts,n_fbins=1024)
    tfr_wvd, t_wvd, f_wvd = wvd.run()

    return tfr_wvd, t_wvd , ts ,dt




In [ ]:
def plot_WVD_log(tfr_wvd, t_wvd , ts, dt, sqmod ,thr):
    f_wvd=np.linspace(0,(1/dt)/2,num=tfr_wvd.shape[0])
    df_wvd = f_wvd[1]-f_wvd[0]  # the frequency step in the WVT
    """print(f_wvd)
    print(f_wvd.shape)
    print("fs ",1/dt)"""

    if(sqmod==True):
      sqmod_tfr= np.abs(tfr_wvd)**2
    else:
      sqmod_tfr= np.abs(tfr_wvd)

    reference = np.median(sqmod_tfr)
    tfr_new = sqmod_tfr/reference
    tfr_log = 10 * np.log10(tfr_new)

    ##threshold
    if(thr>0):
      threshold = np.amax(tfr_log) * thr
      tfr_log[tfr_log <= threshold] = 0.0  

        
    #threshold = np.amax(tfr_log) * thr
    #tfr_log[tfr_log <= threshold] = 0.0

    """ print(tfr_new.shape)
    print(tfr_log.shape)"""
    end= int(tfr_log.shape[0]/2)
    #tfr_log[0:end,:]
    #[0:end,:]
    im = plt.imshow(tfr_log[0:end,:], aspect='auto', origin='lower',
           extent=(ts[0] - dt/2, ts[-1] + dt/2,
                   f_wvd[0]-df_wvd/2, f_wvd[-1]+df_wvd/2),cmap="jet",
                   vmin=np.min(tfr_log), vmax=np.max(tfr_log))

    #plt.xlabel('time [s]')
    #plt.ylabel('frequency [Hz]')
    #cbar = plt.colorbar(im)
    plt.axis('off')
    plt.margins(y=0,x=0)

In [ ]:
animal = "ForestBG"
#Finalize_dataset_Aug2
folder_path_train="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Environment data\Environment data augmentation_split\\train"
folder_path_val="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Environment data\Environment data augmentation_split\\val"
folder_path_test="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Environment data\Environment data augmentation_split\\test"
dest_path_train ="D:\FYP dataset new\MMU folder_FYP_WVD creation\Data version 5 threshold = 0\\train\\" + animal
dest_path_test ="D:\FYP dataset new\MMU folder_FYP_WVD creation\Data version 5 threshold = 0\\test\\" + animal
dest_path_val ="D:\FYP dataset new\MMU folder_FYP_WVD creation\Data version 5 threshold = 0\\val\\" + animal
#dest_path_rest ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\\rest\\" + animal


train_list=os.listdir(folder_path_train)
val_list=os.listdir(folder_path_val)
test_list=os.listdir(folder_path_test)

In [ ]:
audio_list=os.listdir(audio_path)
low_c = 1
high_c = 5000

for i in range(120,240):
    
    #len(audio_list)
    if train_list[i] ==".ipynb_checkpoints":
        continue
        
    """elif(audio_list[i][:5] =="shift"):
        continue"""
    audio_ori_path = folder_path_train+"\\"+train_list[i]
    samplerate, data ,length= read_audio(audio_ori_path)
  
    filter_data=filter_signal(data,samplerate,low_c,high_c,2)
    downsample_data= downsampling(filter_data,samplerate, 32000)
    
    """plt.figure(figsize=(7, 5))
    spectrogram_value_clean_trim,min_db,max_db=spectrogram(downsample_data,downsample_data.shape[0],length,samplerate)
    plt.title("Spectrogram (downsampling)  "+audio_list[i])"""

    plt.figure(figsize=(7, 5))
    tfr_wvd,t_wvd ,ts, dt = WVD(downsample_data,samplerate,downsample_data.shape[0],length)
    plot_WVD_log(tfr_wvd,t_wvd, ts, dt, True , 0)

    new_name=train_list[i][:-4]+".png"
    plotpath=dest_path_train + '/'+new_name
  

In [ ]:
import time

In [ ]:
start = time.time()
audio_ori_path = "Augmentation Sound/Clean/Tapir_clean/wild_10_2_ch0.wav"
samplerate, data ,length= read_audio(audio_ori_path)

filter_data=filter_signal(data,samplerate,500,17000,2)
downsample_data= downsampling(filter_data,samplerate, 32000)

plt.figure(figsize=(7, 5))
tfr_wvd,t_wvd ,ts, dt = WVD(downsample_data,samplerate,downsample_data.shape[0],length)
plot_WVD_log(tfr_wvd,t_wvd, ts, dt, True , 0)

end = time.time()
duration = end - start
print("duration : ",duration)